In [1]:
import functools
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
import datasets
from datasets import Dataset
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm
import transformers
from transformers_interpret import SequenceClassificationExplainer

In [2]:
cols = ['label', 'text']
df = pd.read_csv('../data/greek_fake_news.csv')
df.rename(columns = {"is_fake": "label"}, inplace = True)

raw_datasets = Dataset.from_pandas(df[cols])
raw_datasets = raw_datasets.train_test_split(train_size = 0.8)


In [3]:
#transformer_name = 'nlpaueb/bert-base-greek-uncased-v1'
transformer_name = 'gealexandri/greeksocialbert-base-greek-uncased-v1'

tokenizer = transformers.AutoTokenizer.from_pretrained(transformer_name)

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=512, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [5]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 80
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 20
    })
})

In [6]:
small_train_dataset = tokenized_datasets["train"]
small_eval_dataset = tokenized_datasets["test"]
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(transformer_name, num_labels=2)

Some weights of the model checkpoint at gealexandri/greeksocialbert-base-greek-uncased-v1 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoin

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir = "model",
                                  overwrite_output_dir=True,
                                  per_device_train_batch_size = 4,
                                  per_device_eval_batch_size = 4,
                                  num_train_epochs= 10)

In [9]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 80
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 200


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
txt = """
Την γεωπολιτική διάσταση του ενεργειακού και τον ρόλο που μπορεί να παίξει η Ανατολική Μεσόγειος ως εναλλακτική πηγή στην ενεργειακή ασφάλεια της Ευρώπης ανέδειξε σύμφωνα με κυβερνητικές πηγές ο Κυριάκος Μητσοτάκης στη σχετική συζήτηση στο Ευρωπαϊκό Συμβούλιο.

Ο πρωθυπουργός, σημειώνουν οι ίδιες πηγές, υπογράμμισε ότι τόσο στην ηλεκτρική ενέργεια όσο και στο φυσικό αέριο, η Αίγυπτος, με την οποία η χώρα μας υπέγραψε συμφωνία ηλεκτρικής διασύνδεσης, μπορεί να γίνει η γέφυρα της Αφρικής με την Ευρώπη, καθώς μπορεί να εξάγει ηλεκτρική πράσινη ενέργεια σε χαμηλές τιμές και να τη μεταφέρει μέσω Ελλάδας στην Ευρώπη, αλλά και υγροποιημένο φυσικό αέριο.
Επιπλέον οι ίδιες πηγές αναφέρουν ότι ο κ. Μητσοτάκης τόνισε την ανάγκη «να μελετηθεί η ευρωπαϊκή αγορά ενέργειας προκειμένου να εξεταστεί τί παρεμβάσεις μπορούν να γίνουν για να μην είμαστε εκτεθειμένοι σε απότομες μεταβολές τιμών». «Πρέπει να κάνουμε ό,τι περνάει απ’ το χέρι μας για να μην μεταφερθεί η κρίση στα νοικοκυριά», υπογράμμισε.
"""

cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
word_attributions = cls_explainer(txt)

cls_explainer.visualize()